##### Este script consolida todos los archivos de meteorología y pivotea los valores para cada hora, lo anterior para tener registros organizados por fila para cada punto de muestreo junto la fecha hora y su respectivo valor


In [ ]:
import os
import pandas as pd

# Definir la ruta al directorio que contiene los archivos CSV
directory_path = 'datos\\Metereologia'

# Inicializar un DataFrame vacío para la consolidación
consolidado_metereologico = pd.DataFrame()

# Iterar sobre cada archivo en el directorio
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        # Extraer el mes y el año del nombre del archivo
        parts = filename.split('-')
        mes = parts[0]
        año = parts[-1].split('.')[0]

        # Leer el archivo CSV en un DataFrame, especificando ';' como separador
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, sep=';')

        # Verificar si las columnas coinciden con el DataFrame consolidado
        if not consolidado_metereologico.empty:
            if len(df.columns) != len(consolidado_metereologico.columns):
                print(f"Column mismatch detected in file: {filename}. Please review the file.")
                continue
            if not all(df.columns == consolidado_metereologico.columns):
                print(f"Column mismatch detected in file: {filename}. Please review the file.")
                continue

        # Agregar los datos al DataFrame consolidado
        consolidado_metereologico = pd.concat([consolidado_metereologico, df], ignore_index=True)

# Imprimir el DataFrame consolidado
consolidado_metereologico

In [19]:
#ORder the DataFrame by 'DIA', 'MES', and 'ANO' columns
consolidado_metereologico['DIA'] = consolidado_metereologico['DIA'].astype(int)
consolidado_metereologico['MES'] = consolidado_metereologico['MES'].astype(int)
consolidado_metereologico['ANO'] = consolidado_metereologico['ANO'].astype(int)
consolidado_metereologico = consolidado_metereologico.sort_values(by=['ANO', 'MES', 'DIA']).reset_index(drop=True)
consolidado_metereologico.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2019,1,1,0.65,V,...,0.43,V,0.58,V,0.38,V,0.67,V,0.18,V
1,28,79,102,82,28079102_82_98,2019,1,1,64.00,V,...,253.00,V,277.00,V,69.00,V,40.00,V,333.00,V
2,28,79,102,83,28079102_83_98,2019,1,1,3.10,V,...,7.10,V,6.60,V,5.80,V,4.10,V,3.40,V
3,28,79,102,86,28079102_86_98,2019,1,1,50.00,V,...,43.00,V,47.00,V,49.00,V,54.00,V,55.00,V
4,28,79,102,87,28079102_87_98,2019,1,1,949.00,V,...,949.00,V,949.00,V,949.00,V,949.00,V,949.00,V


In [ ]:
# Valores a filtrar para la columna MAGNITUD
valores_comparacion = [83, 89]

# Eliminar filas de codigos diferentes a 83 "TEMPERATURA" y 89 "PRECIPITACIÓN" para las columna MAGNITUD
consolidado_metereologico = consolidado_metereologico[consolidado_metereologico['MAGNITUD'].isin(valores_comparacion)]

In [22]:
consolidado_metereologico.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
2,28,79,102,83,28079102_83_98,2019,1,1,3.1,V,...,7.1,V,6.6,V,5.8,V,4.1,V,3.4,V
6,28,79,102,89,28079102_89_98,2019,1,1,0.0,V,...,0.0,V,0.0,V,0.0,V,0.0,V,0.0,V
9,28,79,103,83,28079103_83_98,2019,1,1,-1.1,V,...,3.1,V,1.6,V,0.6,V,-0.4,V,-1.1,V
13,28,79,103,89,28079103_89_98,2019,1,1,0.0,V,...,0.0,V,0.0,V,0.0,V,0.0,V,0.0,V
16,28,79,104,83,28079104_83_98,2019,1,1,-0.8,V,...,3.0,V,2.1,V,0.8,V,0.3,V,-0.6,V


In [ ]:
# Melt the DataFrame to pivot the columns H01, H02, etc. and V01, V02, etc.
melted_h = pd.melt(consolidado_metereologico, 
                   id_vars=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'], 
                   value_vars=[col for col in consolidado_metereologico.columns if col.startswith('H')],
                   var_name='HORA',
                   value_name='VALOR')

melted_v = pd.melt(consolidado_metereologico, 
                   id_vars=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'], 
                   value_vars=[col for col in consolidado_metereologico.columns if col.startswith('V')],
                   var_name='HORA_V',
                   value_name='VERIF')

# Extract the hour from the HORA column and format it as HH:00
melted_h['HORA'] = melted_h['HORA'].str.extract('(\d+)')[0].astype(int).astype(str).str.zfill(2) + ':00'
melted_v['HORA_V'] = melted_v['HORA_V'].str.extract('(\d+)')[0].astype(int).astype(str).str.zfill(2) + ':00'

# Merge the two melted DataFrames on the common columns and the hour columns
final_df = pd.merge(melted_h, melted_v, left_on=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA', 'HORA'], right_on=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA', 'HORA_V'])

# Drop the redundant HORA_V column
final_df = final_df.drop(columns=['HORA_V'])

# Modificar la columna 'PUNTO_MUESTREO' para eliminar el sufijo '_98'
final_df['PUNTO_MUESTREO'] = final_df['PUNTO_MUESTREO'].str.replace('_98', '')

In [30]:
# eliminar columnas innecesarias
columnas_a_eliminar = ['PROVINCIA', 'MUNICIPIO','ESTACION', 'MAGNITUD']
# Eliminar las columnas del DataFrame
final_df.drop(columnas_a_eliminar, axis=1, inplace=True)
final_df.head()

,PUNTO_MUESTREO,ANO,MES,DIA,HORA,VALOR,VERIF
0,28079102_83,2019,1,1,01:00,3.1,V
1,28079102_89,2019,1,1,01:00,0.0,V
2,28079103_83,2019,1,1,01:00,-1.1,V
3,28079103_89,2019,1,1,01:00,0.0,V
4,28079104_83,2019,1,1,01:00,-0.8,V


In [31]:
# Guardar el DataFrame final en un archivo CSV
output_file_path = 'datos\\consolidado_metereologico.csv'
final_df.to_csv(output_file_path, index=False, sep=';')